<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Scale-and-Average-Readability-Scores" data-toc-modified-id="Scale-and-Average-Readability-Scores-1">Scale and Average Readability Scores</a></span></li><li><span><a href="#Lag-Scores" data-toc-modified-id="Lag-Scores-2">Lag Scores</a></span></li><li><span><a href="#Volume-Count" data-toc-modified-id="Volume-Count-3">Volume Count</a></span></li><li><span><a href="#Summarize-by-Day-for-each-Stock" data-toc-modified-id="Summarize-by-Day-for-each-Stock-4">Summarize by Day for each Stock</a></span></li><li><span><a href="#Merge-Finalized-Data-with-Return-Data" data-toc-modified-id="Merge-Finalized-Data-with-Return-Data-5">Merge Finalized Data with Return Data</a></span></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
nlp_df = pd.read_csv("nlp_apr2021.csv").drop(["FleschReadingEase","words", "complex_words"], axis=1)# update with finalized data
nlp_df.head(5)

,year,month,day,website,url,pos_mean,pos_median,pos_min,pos_max,neg_mean,...,net_max,Kincaid,ARI,Coleman-Liau,GunningFogIndex,LIX,SMOGIndex,RIX,DaleChallIndex,tags
0,2021,4,1,yahoo.com,https://finance.yahoo.com/news/glancy-prongay-...,0.0,0.0,0.0,0.0,0.21,...,-0.0,19.56,24.56,11.75,24.85,72.81,17.66,12.17,13.74,NWSA
1,2021,4,1,yahoo.com,https://finance.yahoo.com/news/glancy-prongay-...,0.0,0.0,0.0,0.0,0.21,...,-0.0,19.56,24.56,11.75,24.85,72.81,17.66,12.17,13.74,NWS
2,2021,4,1,yahoo.com,https://finance.yahoo.com/news/glancy-prongay-...,0.0,0.0,0.0,0.0,0.21,...,-0.0,19.56,24.56,11.75,24.85,72.81,17.66,12.17,13.74,TWTR
3,2021,4,1,yahoo.com,https://finance.yahoo.com/news/glancy-prongay-...,0.0,0.0,0.0,0.0,0.21,...,-0.0,19.56,24.56,11.75,24.85,72.81,17.66,12.17,13.74,WAT
4,2021,4,1,yahoo.com,https://news.yahoo.com/covid-19-death-confirme...,0.3,0.0,0.0,1.0,0.09,...,1.0,9.29,11.97,11.57,13.60,46.70,12.10,5.38,11.78,BRO


## Scale and Average Readability Scores

In [3]:
nlp_df.columns

Index(['year', 'month', 'day', 'website', 'url', 'pos_mean', 'pos_median',
       'pos_min', 'pos_max', 'neg_mean', 'neg_median', 'neg_min', 'neg_max',
       'neu_mean', 'neu_median', 'neu_min', 'neu_max', 'net_mean',
       'net_median', 'net_min', 'net_max', 'Kincaid', 'ARI', 'Coleman-Liau',
       'GunningFogIndex', 'LIX', 'SMOGIndex', 'RIX', 'DaleChallIndex', 'tags'],
      dtype='object')

In [4]:
read_cols = ['Kincaid', 'ARI', 'Coleman-Liau',
             'GunningFogIndex', 'LIX', 'SMOGIndex', 'RIX', 'DaleChallIndex']

In [5]:
scaler = MinMaxScaler()
scaled_df = scaler.fit_transform(nlp_df[read_cols].to_numpy())
scaled_df = pd.DataFrame(scaled_df, columns=read_cols)
scaled_df.head()

,Kincaid,ARI,Coleman-Liau,GunningFogIndex,LIX,SMOGIndex,RIX,DaleChallIndex
0,1.000000,1.000000,0.737071,1.00000,1.000000,1.000000,1.000000,1.000000
1,1.000000,1.000000,0.737071,1.00000,1.000000,1.000000,1.000000,1.000000
2,1.000000,1.000000,0.737071,1.00000,1.000000,1.000000,1.000000,1.000000
3,1.000000,1.000000,0.737071,1.00000,1.000000,1.000000,1.000000,1.000000
4,0.528899,0.518915,0.727273,0.45757,0.511506,0.536667,0.415663,0.454039


In [6]:
scaled_df['read_score'] = scaled_df[read_cols].mean(axis=1)
scaled_df = scaled_df.drop(read_cols,axis=1)
scaled_df.head()

,read_score
0,0.967134
1,0.967134
2,0.967134
3,0.967134
4,0.518816


In [7]:
scaled_df = nlp_df.join(scaled_df).drop(read_cols,axis=1)
scaled_df.head()

,year,month,day,website,url,pos_mean,pos_median,pos_min,pos_max,neg_mean,...,neu_mean,neu_median,neu_min,neu_max,net_mean,net_median,net_min,net_max,tags,read_score
0,2021,4,1,yahoo.com,https://finance.yahoo.com/news/glancy-prongay-...,0.0,0.0,0.0,0.0,0.21,...,0.79,1.0,0.0,1.0,-0.21,-0.0,-1.00,-0.0,NWSA,0.967134
1,2021,4,1,yahoo.com,https://finance.yahoo.com/news/glancy-prongay-...,0.0,0.0,0.0,0.0,0.21,...,0.79,1.0,0.0,1.0,-0.21,-0.0,-1.00,-0.0,NWS,0.967134
2,2021,4,1,yahoo.com,https://finance.yahoo.com/news/glancy-prongay-...,0.0,0.0,0.0,0.0,0.21,...,0.79,1.0,0.0,1.0,-0.21,-0.0,-1.00,-0.0,TWTR,0.967134
3,2021,4,1,yahoo.com,https://finance.yahoo.com/news/glancy-prongay-...,0.0,0.0,0.0,0.0,0.21,...,0.79,1.0,0.0,1.0,-0.21,-0.0,-1.00,-0.0,WAT,0.967134
4,2021,4,1,yahoo.com,https://news.yahoo.com/covid-19-death-confirme...,0.3,0.0,0.0,1.0,0.09,...,0.61,1.0,0.0,1.0,0.21,-0.0,-0.99,1.0,BRO,0.518816


## Lag Scores

## Volume Count

In [8]:
count_df = pd.DataFrame(scaled_df.groupby(["year", "month", "day", "tags"])["website"]
                        .count()).reset_index().rename(columns={"website":"volume"})
count_df.head()

,year,month,day,tags,volume
0,2021,4,1,AEP,1
1,2021,4,1,AWK,1
2,2021,4,1,BRO,1
3,2021,4,1,CFG,1
4,2021,4,1,COF,2


## Summarize by Day for each Stock

In [9]:
scaled_df.columns

Index(['year', 'month', 'day', 'website', 'url', 'pos_mean', 'pos_median',
       'pos_min', 'pos_max', 'neg_mean', 'neg_median', 'neg_min', 'neg_max',
       'neu_mean', 'neu_median', 'neu_min', 'neu_max', 'net_mean',
       'net_median', 'net_min', 'net_max', 'tags', 'read_score'],
      dtype='object')

In [10]:
summary_cols = ['pos_mean', 'pos_median',
       'pos_min', 'pos_max', 'neg_mean', 'neg_median', 'neg_min', 'neg_max',
       'neu_mean', 'neu_median', 'neu_min', 'neu_max', 'net_mean',
       'net_median', 'net_min', 'net_max', 'read_score']

In [11]:
summary_df = pd.DataFrame(scaled_df.groupby(["year", "month", "day", "tags"])[summary_cols]
                        .mean()).reset_index()
summary_df.head(5)

,year,month,day,tags,pos_mean,pos_median,pos_min,pos_max,neg_mean,neg_median,...,neg_max,neu_mean,neu_median,neu_min,neu_max,net_mean,net_median,net_min,net_max,read_score
0,2021,4,1,AEP,0.43,0.01,0.0,1.0,0.00,0.0,...,0.0,0.57,0.99,0.0,1.0,0.43,0.01,0.00,1.0,0.753541
1,2021,4,1,AWK,0.43,0.01,0.0,1.0,0.00,0.0,...,0.0,0.57,0.99,0.0,1.0,0.43,0.01,0.00,1.0,0.753541
2,2021,4,1,BRO,0.30,0.00,0.0,1.0,0.09,0.0,...,1.0,0.61,1.00,0.0,1.0,0.21,0.00,-0.99,1.0,0.518816
3,2021,4,1,CFG,0.05,0.00,0.0,1.0,0.24,0.0,...,1.0,0.71,1.00,0.0,1.0,-0.19,0.00,-1.00,1.0,0.639720
4,2021,4,1,COF,0.20,0.00,0.0,1.0,0.00,0.0,...,0.1,0.80,1.00,0.0,1.0,0.20,0.00,-0.10,1.0,0.881164


In [12]:
final_df = count_df.merge(summary_df, 
              how="left", 
              on=["year", "month", "day", "tags"])
final_df

,year,month,day,tags,volume,pos_mean,pos_median,pos_min,pos_max,neg_mean,...,neg_max,neu_mean,neu_median,neu_min,neu_max,net_mean,net_median,net_min,net_max,read_score
0,2021,4,1,AEP,1,0.430000,0.01,0.0,1.000000,0.000000,...,0.000000,0.57,0.99,0.000000,1.0,0.43,0.01,0.000000,1.000000,0.753541
1,2021,4,1,AWK,1,0.430000,0.01,0.0,1.000000,0.000000,...,0.000000,0.57,0.99,0.000000,1.0,0.43,0.01,0.000000,1.000000,0.753541
2,2021,4,1,BRO,1,0.300000,0.00,0.0,1.000000,0.090000,...,1.000000,0.61,1.00,0.000000,1.0,0.21,0.00,-0.990000,1.000000,0.518816
3,2021,4,1,CFG,1,0.050000,0.00,0.0,1.000000,0.240000,...,1.000000,0.71,1.00,0.000000,1.0,-0.19,0.00,-1.000000,1.000000,0.639720
4,2021,4,1,COF,2,0.200000,0.00,0.0,1.000000,0.000000,...,0.100000,0.80,1.00,0.000000,1.0,0.20,0.00,-0.100000,1.000000,0.881164
5,2021,4,1,COST,1,0.500000,0.50,0.0,1.000000,0.000000,...,0.000000,0.50,0.50,0.000000,1.0,0.50,0.50,0.000000,1.000000,0.848621
6,2021,4,1,CVX,1,0.000000,0.00,0.0,0.000000,0.090000,...,1.000000,0.91,1.00,0.000000,1.0,-0.09,0.00,-1.000000,0.000000,0.122911
7,2021,4,1,DOW,1,0.000000,0.00,0.0,0.000000,0.090000,...,1.000000,0.91,1.00,0.000000,1.0,-0.09,0.00,-1.000000,0.000000,0.122911
8,2021,4,1,FBHS,1,0.500000,0.50,0.0,1.000000,0.000000,...,0.000000,0.50,0.50,0.000000,1.0,0.50,0.50,0.000000,1.000000,0.848621
9,2021,4,1,FDS,1,0.000000,0.00,0.0,0.000000,0.090000,...,1.000000,0.91,1.00,0.000000,1.0,-0.09,0.00,-1.000000,0.000000,0.122911


## Merge Finalized Data with Return Data

In [13]:
return_df = pd.read_csv("../00-data/stock_returns.csv",index_col=[0])
return_df.head(5)
df = final_df.merge(return_df, 
               how="left", 
               left_on=["year", "month", "day", "tags"],
               right_on=["year", "month", "day", "ticker"]).drop("tags",axis=1)
first_col = df.pop("ticker")
df.insert(0,'ticker',first_col)
df

,ticker,year,month,day,volume,pos_mean,pos_median,pos_min,pos_max,neg_mean,...,net_mean,net_median,net_min,net_max,read_score,same_day_raw,next_day_raw,sp_avg_return,same_day_relative,next_day_relative
0,AEP,2021,4,1,1,0.430000,0.01,0.0,1.000000,0.000000,...,0.43,0.01,0.000000,1.000000,0.753541,-0.753827,-1.255574,-0.606019,-0.147808,0.820586
1,AWK,2021,4,1,1,0.430000,0.01,0.0,1.000000,0.000000,...,0.43,0.01,0.000000,1.000000,0.753541,0.033194,-1.171222,-0.606019,0.639213,0.108078
2,BRO,2021,4,1,1,0.300000,0.00,0.0,1.000000,0.090000,...,0.21,0.00,-0.990000,1.000000,0.518816,-1.549878,0.171082,-0.606019,-0.943859,-0.702937
3,CFG,2021,4,1,1,0.050000,0.00,0.0,1.000000,0.240000,...,-0.19,0.00,-1.000000,1.000000,0.639720,-1.904764,1.340070,-0.606019,-1.298745,0.141522
4,COF,2021,4,1,2,0.200000,0.00,0.0,1.000000,0.000000,...,0.20,0.00,-0.100000,1.000000,0.881164,-1.066891,-0.943975,-0.606019,-0.460872,1.141379
5,COST,2021,4,1,1,0.500000,0.50,0.0,1.000000,0.000000,...,0.50,0.50,0.000000,1.000000,0.848621,-0.680772,-0.928673,-0.606019,-0.074753,-0.605994
6,CVX,2021,4,1,1,0.000000,0.00,0.0,0.000000,0.090000,...,-0.09,0.00,-1.000000,0.000000,0.122911,-0.465512,1.153883,-0.606019,0.140507,0.368009
7,DOW,2021,4,1,1,0.000000,0.00,0.0,0.000000,0.090000,...,-0.09,0.00,-1.000000,0.000000,0.122911,-1.016414,0.630775,-0.606019,-0.410395,-0.473738
8,FBHS,2021,4,1,1,0.500000,0.50,0.0,1.000000,0.000000,...,0.50,0.50,0.000000,1.000000,0.848621,-0.966536,0.488503,-0.606019,-0.360517,-0.006596
9,FDS,2021,4,1,1,0.000000,0.00,0.0,0.000000,0.090000,...,-0.09,0.00,-1.000000,0.000000,0.122911,-1.912818,-0.435513,-0.606019,-1.306799,1.035818


In [14]:
df.to_excel("agg_data.xlsx",index=False)